In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/aviachipta-narxini-bashorat-qilish/train_data.csv
/kaggle/input/aviachipta-narxini-bashorat-qilish/test_data.csv
/kaggle/input/aviachipta-narxini-bashorat-qilish/sample_solution.csv


In [2]:
import numpy as np 
import pandas as pd 
import sklearn
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error


In [3]:
data = pd.read_csv('/kaggle/input/aviachipta-narxini-bashorat-qilish/train_data.csv', index_col=0)
test_data = pd.read_csv('/kaggle/input/aviachipta-narxini-bashorat-qilish/test_data.csv', index_col=0)
data.head()

,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price
id,,,,,,,,,,,
1,Vistara,UK-810,Bangalore,Early_Morning,one,Night,Mumbai,Economy,14.25,21,7212
2,SpiceJet,SG-5094,Hyderabad,Evening,zero,Night,Kolkata,Economy,1.75,7,5292
3,Vistara,UK-846,Bangalore,Morning,one,Evening,Delhi,Business,9.58,5,60553
4,Vistara,UK-706,Kolkata,Morning,one,Evening,Hyderabad,Economy,6.75,28,5760
5,Indigo,6E-5394,Chennai,Early_Morning,zero,Morning,Mumbai,Economy,2.00,4,10712


In [4]:
X = data.drop(columns='price', axis=1)
y = data['price'].copy()

In [5]:
data.shape

(20000, 11)

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20000 entries, 1 to 20000
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   airline           20000 non-null  object 
 1   flight            20000 non-null  object 
 2   source_city       20000 non-null  object 
 3   departure_time    20000 non-null  object 
 4   stops             20000 non-null  object 
 5   arrival_time      20000 non-null  object 
 6   destination_city  20000 non-null  object 
 7   class             20000 non-null  object 
 8   duration          20000 non-null  float64
 9   days_left         20000 non-null  int64  
 10  price             20000 non-null  int64  
dtypes: float64(1), int64(2), object(8)
memory usage: 1.8+ MB


In [7]:
data['flight'].value_counts()

flight
UK-706     208
UK-720     190
UK-822     178
UK-836     169
UK-772     169
          ... 
6E-2471      1
6E-821       1
SG-1031      1
6E-759       1
6E-7201      1
Name: count, Length: 1310, dtype: int64

In [8]:
X.drop(columns='flight', inplace=True)

In [9]:
text_cols=['airline',  'source_city', 'departure_time','stops', 'arrival_time', 'destination_city', 'class']
num_cols=['duration', 'days_left']

label_encod = LabelEncoder()
scaler=StandardScaler()

label_encoders={}

for col in text_cols:
    X[col]=label_encod.fit_transform(X[col])
    label_encoders[col]=label_encod

X.head()

,airline,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left
id,,,,,,,,,
1,5,0,1,0,5,5,1,14.25,21
2,4,3,2,2,5,4,1,1.75,7
3,5,0,4,0,2,2,0,9.58,5
4,5,4,4,0,2,3,1,6.75,28
5,3,1,1,2,4,5,1,2.00,4


In [10]:
X['duration']=X['duration'].astype(float)

In [11]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20000 entries, 1 to 20000
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   airline           20000 non-null  int64  
 1   source_city       20000 non-null  int64  
 2   departure_time    20000 non-null  int64  
 3   stops             20000 non-null  int64  
 4   arrival_time      20000 non-null  int64  
 5   destination_city  20000 non-null  int64  
 6   class             20000 non-null  int64  
 7   duration          20000 non-null  float64
 8   days_left         20000 non-null  int64  
dtypes: float64(1), int64(8)
memory usage: 1.5 MB


In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, y_test.shape

((16000, 9), (4000,))

In [13]:
X_train_prepared = scaler.fit_transform(X_train)
X_train_prepared

array([[-0.6109521 , -1.47043494, -0.24285189, ...,  0.67361153,
        -1.02542705,  0.44421574],
       [-1.15784892,  0.2345819 , -0.81150845, ..., -1.48453515,
         0.68489074,  0.95782886],
       [ 1.02973834,  0.2345819 , -1.38016501, ...,  0.67361153,
        -0.58283869, -1.09662363],
       ...,
       [-1.15784892, -0.33375705,  0.89446123, ...,  0.67361153,
         2.2206863 , -1.46349015],
       [-0.6109521 , -1.47043494, -0.81150845, ...,  0.67361153,
        -0.349677  , -0.65638381],
       [-0.06405529,  1.3712598 ,  0.89446123, ...,  0.67361153,
        -1.07150092, -0.94987703]])

In [14]:
LR_model = LinearRegression()

In [15]:
LR_model.fit(X_train_prepared, y_train)

LinearRegression()

In [16]:
sample_data = X_train.sample(10)
sample_data_prepared = scaler.fit_transform(sample_data)

In [17]:
y_sample = y_train.loc[sample_data.index]
y_sample

id
14131    10962
16542    82989
13412     4262
8315     11488
788       6075
1051     54684
13523     5102
13208     5103
7909      4262
18100     7958
Name: price, dtype: int64

In [18]:
predict = LR_model.predict(sample_data_prepared)

In [19]:
pd.DataFrame({"asl narx":y_sample, "bashorat":predict})

,asl narx,bashorat
id,,
14131,10962,12309.573391
16542,82989,66053.356768
13412,4262,10681.596881
8315,11488,13436.518097
788,6075,9917.539290
1051,54684,58592.049868
13523,5102,7368.435708
13208,5103,5185.972698
7909,4262,10210.467908


In [20]:
X_test_prepared = scaler.fit_transform(X_test)

In [21]:
y_pred=LR_model.predict(X_test_prepared)

In [22]:
mean_absolute_error(y_test, y_pred)

4623.773277263513

In [23]:
RF_model = RandomForestRegressor()

In [24]:
RF_model.fit(X_train_prepared, y_train)

RandomForestRegressor()

In [25]:
y_pred = RF_model.predict(X_test_prepared)
mean_absolute_error(y_test, y_pred)

2082.842254672619

In [26]:
test_data.head()

,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left
id,,,,,,,,,,
1,Air_India,AI-765,Kolkata,Evening,one,Night,Delhi,Business,28.25,2
2,Vistara,UK-747,Delhi,Early_Morning,one,Night,Mumbai,Business,13.83,34
3,Air_India,AI-570,Mumbai,Early_Morning,zero,Early_Morning,Chennai,Business,2.00,30
4,AirAsia,I5-974,Hyderabad,Night,one,Late_Night,Delhi,Economy,5.17,26
5,Air_India,AI-770,Kolkata,Night,one,Afternoon,Mumbai,Economy,16.33,35


In [27]:
text_cols=['airline',  'source_city', 'departure_time','stops', 'arrival_time', 'destination_city', 'class']
num_cols=['duration', 'days_left']

label_encod = LabelEncoder()
scaler=StandardScaler()

label_encoders={}

for col in text_cols:
    test_data[col]=label_encod.fit_transform(test_data[col])
    label_encoders[col]=label_encod

test_data.head()

,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left
id,,,,,,,,,,
1,1,AI-765,4,2,0,5,2,0,28.25,2
2,5,UK-747,2,1,0,5,5,0,13.83,34
3,1,AI-570,5,1,2,1,1,0,2.00,30
4,0,I5-974,3,5,0,3,2,1,5.17,26
5,1,AI-770,4,5,0,0,5,1,16.33,35


In [28]:
test_data.drop(columns='flight', inplace=True)

In [29]:
test_data['duration']=test_data['duration'].astype(float)

In [30]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5000 entries, 1 to 5000
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   airline           5000 non-null   int64  
 1   source_city       5000 non-null   int64  
 2   departure_time    5000 non-null   int64  
 3   stops             5000 non-null   int64  
 4   arrival_time      5000 non-null   int64  
 5   destination_city  5000 non-null   int64  
 6   class             5000 non-null   int64  
 7   duration          5000 non-null   float64
 8   days_left         5000 non-null   int64  
dtypes: float64(1), int64(8)
memory usage: 390.6 KB


In [31]:
test_data_prepared = scaler.fit_transform(test_data)

In [32]:
Predict_data = RF_model.predict(test_data_prepared)

In [33]:
result = pd.DataFrame({"BAshorat narx":Predict_data})

In [34]:
result.to_csv("result_data", index=False)